In [2]:
import jax.numpy as jnp
import jax
from jax import random
import matplotlib.pyplot as plt
from jax import lax

In [12]:
Spot = 36   # stock price
k = 12   # Polynmial order

Spot = 36   # stock price
σ = 0.2     # stock volatility
K = 40      # strike price
r = 0.06    # risk free rate
n = 100000  # Number of simualted paths
m = 50      # Number of exercise dates
T = 1       # maturity
order = 12   # Polynmial order
dt = T / m

S0 = jnp.ones((1,n))*Spot
discount = jnp.exp(-r * dt)


# Random price generation

def Prices(S0,m):
  dB = jnp.sqrt(dt) * jax.random.normal(key=random.PRNGKey(m), shape=(1,jnp.size(S0)))[0]
  S0= S0 + r * S0 * dt + σ * S0 * dB
  
  return S0, S0
SF, S_list = lax.scan(Prices, S0, jnp.arange(m))
S= jnp.stack(S_list)[:,0]


##################################################

def payoff_put(S):
    return jnp.maximum(K - S, 0.)

##################################################

# Scaling prices

def scale(S0,m):
  dB = jnp.sqrt(dt) * jax.random.normal(key=random.PRNGKey(m), shape=(1,jnp.size(S0)))[0]
  S0= S0 + r * S0 * dt + σ * S0 * dB
  xmin = S0.min(axis = 1)
  xmax = S0.max(axis = 1)
  a = 2 / (xmax - xmin)
  b = 1 - a * xmax
  c = a *S0 + b
  return S0, c
f, c = lax.scan(scale, S0, jnp.arange(m))
Sc= jnp.stack(c)[:,0]

###################################################

B = jnp.array([jnp.ones(len(Sc[0])), Sc[0]])
B0 = jnp.array(Sc[0])


def payoff_put(S):
    return jnp.maximum(K - S, 0.)



###################################################

def compute(S0,m):
  dB = jnp.sqrt(dt) * jax.random.normal(key=random.PRNGKey(m), shape=(1,jnp.size(S0)))[0]
  S0= S0 + r * S0 * dt + σ * S0 * dB
  xmin = S0.min(axis = 1)
  xmax = S0.max(axis = 1)
  a = 2 / (xmax - xmin)
  b = 1 - a * xmax
  c = a *S0 + b
  #######################################
  def Che(B, k):
    C1 = B[0]
    C2 = B[1]
    C3 = 2 * B0 * C2 - C1
    
    B = jnp.array([C2, C3])
    return B, C3
  f, H = lax.scan(Che, B, jnp.ones(k-2))
  jnp.concatenate([B, H], axis=0)
  #######################################
  value_if_exercise = payoff_put(S[-1])
  discounted_future_cashflows = value_if_exercise * discount
  X = jnp.concatenate([B, H], axis=0).T
  y = discounted_future_cashflows
  Θ = jnp.linalg.solve(X.T @ X, X.T @ y)
  Θ
  
  
  value_if_wait = X @ Θ
  
  value_if_exercise = payoff_put(S[-2 - m])
  exercise = value_if_exercise >= value_if_wait
  discounted_future_cashflows = discount * jnp.where(
              exercise,
              value_if_exercise,
              discounted_future_cashflows)
  return S0, discounted_future_cashflows
f, c = lax.scan(compute, S0, jnp.arange(m))

c.mean()


DeviceArray(4.272576, dtype=float32)